<a href="https://colab.research.google.com/github/iakovkh/mypython/blob/main/MyNeural_v40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Configuration Section
data_file_name = '/content/sample_data/NN_cut2.xlsx'
normalize_data = True
split_ratios = (70, 20, 10)  # Train, Test, Control
optimizer_name = 'Adam'  # Other options: 'SGD', 'RMSprop', etc.
loss_function_name = 'MSELoss'  # Other options: 'CrossEntropyLoss', 'L1Loss', etc.
activation_function_name = 'ReLU'  # Other options: 'Sigmoid', 'Tanh', etc.
epochs = 500
network_structure = [(1, 32), (2, 3), (3, 5), (4, 2)]
output_file_path = 'result.xlsx'
network_structure_file = 'neurol_network.txt'

# Section I: General Data Preparation
class MyMatrix:
    def __init__(self):
        self.data = None
        self.train_data = None
        self.test_data = None
        self.control_data = None

    def read_data_from_file(self, file_name):
        self.data = pd.read_excel(file_name)

    def display_console(self, column=None):
        if column:
            print(self.data[column])
        else:
            print(self.data)

    def display_visual(self, column=None):
        if column:
            self.data[column].plot()
            plt.show()
        else:
            self.data.plot(subplots=True)
            plt.show()

    def normalize_data(self):
        self.data = (self.data - self.data.min()) / (self.data.max() - self.data.min())

    def split_data(self, ratios):
        train_size = int(len(self.data) * ratios / 100)
        test_size = int(len(self.data) * ratios / 100)
        self.train_data = self.data[:train_size]
        self.test_data = self.data[train_size:train_size + test_size]
        self.control_data = self.data[train_size + test_size:]

# Section II: Read and Prepare Data
MyData = MyMatrix()
MyData.read_data_from_file(data_file_name)
if normalize_data:
    MyData.normalize_data()
MyData.split_data(split_ratios)

# Section III: Design the Neural Network
class MyNeurolNetwork(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size):
        super(MyNeurolNetwork, self).__init__()
        layers = []
        in_size = input_size
        for layer in hidden_layers:
            layers.append(nn.Linear(in_size, layer))
            layers.append(getattr(nn, activation_function_name)())
            in_size = layer
        layers.append(nn.Linear(in_size, output_size))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

    def display_network_structure(self, file_path):
        with open(file_path, 'w') as f:
            for name, param in self.named_parameters():
                f.write(f"{name}: {param.shape}\n")

# Helper function to create the network
def create_network():
    input_size = MyData.train_data.shape
    hidden_layers = [layer for layer in network_structure if layer != len(network_structure)]
    output_size = 1
    return MyNeurolNetwork(input_size, hidden_layers, output_size)

# Section IV: Forecast
def Market_Forecast(Data, Network):
    criterion = getattr(nn, loss_function_name)()
    optimizer = getattr(optim, optimizer_name)(Network.parameters(), lr=0.001)

    train_data = torch.tensor(Data.train_data.values, dtype=torch.float32)
    test_data = torch.tensor(Data.test_data.values, dtype=torch.float32)
    control_data = torch.tensor(Data.control_data.values, dtype=torch.float32)

    for epoch in range(epochs):
        Network.train()
        optimizer.zero_grad()
        outputs = Network(train_data)
        loss = criterion(outputs, train_data[:, 0].unsqueeze(1))
        loss.backward()
        optimizer.step()

    Network.eval()
    with torch.no_grad():
        forecast = Network(control_data.unsqueeze(0))
        actual = control_data[0, 0].item()

    return forecast.item(), actual

# Section V: Forward Testing
def forward_testing(Data, Network):
    control_data = Data.control_data.values
    train_size = int(len(control_data) * split_ratios / 100)
    test_size = int(len(control_data) * split_ratios / 100)

    results = []

    for i in range(len(control_data)):
        forecast, actual = Market_Forecast(Data, Network)
        results.append({'Forecast': forecast, 'Actual': actual})

        if i % (train_size + test_size) < train_size:
            Data.train_data = Data.train_data.append(Data.control_data.iloc[i])
        else:
            Data.test_data = Data.test_data.append(Data.control_data.iloc[i])
        Data.control_data = Data.control_data.drop(Data.control_data.index[i])
        Network = create_network()

    results_df = pd.DataFrame(results)
    results_df.to_excel(output_file_path, index=False)
    return results_df

# Visualization function
def visualize_forecast(results_df):
    plt.figure(figsize=(10, 5))
    plt.plot(results_df['Forecast'], label='Forecasted Values')
    plt.plot(results_df['Actual'], label='Actual Values')
    plt.xlabel('Time')
    plt.ylabel('Values')
    plt.title('Forecasted vs Actual Values Over Time')
    plt.legend()
    plt.show()

# Create and train the network
MyNetwork = create_network()
results_df = forward_testing(MyData, MyNetwork)
visualize_forecast(results_df)
